In [1]:
import random
from dataclasses import dataclass

from pprint import pprint
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost')
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
db = client.test

In [4]:
test_students = [f'stud{n}' for n in range(100)]

In [7]:
tasks = []
for _ in range(1):
    task = {
        'tg_user_id': random.choice(test_students),
        'task': {
            'workshop': random.randint(1, 3),
            'task': random.randint(1, 5),
            'tg_file_id': random.randint(1, 1000000000),
            'file_name': 'blank',
            'scores': []
        }
    }
    res = db.tasks.insert_one(task)
    tasks.append(task)

In [8]:
res

In [9]:
res.inserted_id

ObjectId('5c66e9dd301c3b0741c05c35')

In [80]:
res.acknowledged

True

In [11]:
db.tasks.find_one({'tg_user_id': 'asdf'}) is None

True

In [36]:
db.tasks.find({'tg_user_id': test_students[4]}).count()

/Users/dropout05/Documents/peer_review_bot/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2

In [46]:
groups = db.tasks.aggregate([
    {'$group': {'_id': "$task", 'count': {'$sum': 1}}},
    {'$sort': {'_id': 1}}
])

In [47]:
for group in groups:
    print(group)
    break

{'_id': {'workshop': 1, 'task': 1, 'tg_file_id': 36788485, 'file': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'file_name': 'blank', 'scores': []}, 'count': 1}


In [57]:
user_task = db.tasks.find_one({'tg_user_id': test_students[4], 'task.task': 3})

In [58]:
db.tasks.update_one({'_id': user_task.get('_id')}, {'$push': {'task.scores': 5}})

In [59]:
db.tasks.find_one({'tg_user_id': test_students[4], 'task.task': 3})

{'_id': ObjectId('5c643d8f52924c093a976a14'),
 'tg_user_id': 'stud4',
 'task': {'workshop': 2,
  'task': 3,
  'tg_file_id': 891314104,
  'file': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [3]:
db = client.admin

In [64]:
@dataclass
class Task:
    workshop: int
    subtask: int
    tg_file_id: str
    file_name: str
    scores: list

@dataclass
class UserTask:
    """Pair user-task"""
    tg_user_id: str
    username: str
    task: Task


In [69]:
ut = UserTask('a', 'a', Task(1, 1, '1', '1', []))

In [72]:
asdict(ut)

{'tg_user_id': 'a',
 'username': 'a',
 'task': {'workshop': 1,
  'subtask': 1,
  'tg_file_id': '1',
  'file_name': '1',
  'scores': []}}

In [14]:
pprint(db.command('serverStatus'))

{'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 0, 'warning': 0},
 'connections': {'available': 5732, 'current': 2, 'totalCreated': 5},
 'extra_info': {'note': 'fields vary by platform', 'page_faults': 4422},
 'freeMonitoring': {'state': 'undecided'},
 'globalLock': {'activeClients': {'readers': 0, 'total': 13, 'writers': 0},
                'currentQueue': {'readers': 0, 'total': 0, 'writers': 0},
                'totalTime': 2783571000},
 'host': 'Vladislavs-MacBook-Pro.local',
 'localTime': datetime.datetime(2019, 2, 13, 15, 22, 8, 93000),
 'locks': {'Collection': {'acquireCount': {'r': 2200, 'w': 59}},
           'Database': {'acquireCount': {'R': 2, 'W': 8, 'r': 2200, 'w': 67}},
           'Global': {'acquireCount': {'W': 5, 'r': 7262, 'w': 74},
                      'acquireWaitCount': {'W': 1},
                      'timeAcquiringMicros': {'W': 57}}},
 'logicalSessionRecordCache': {'activeSessionsCount': 1,
                               'lastSessionsCollectionJobCu